# Binomial Option Pricing Model

This notebook explains the binomial option pricing model for European call options and provides a Python implementation with visualizations.

## 1. Mathematical Foundation

The binomial model assumes that the price of an underlying asset follows a discrete-time process where in each period, the price can either go up by a factor $u$ or down by a factor $d$.

### Key Parameters

- $S_0$: Initial price of the underlying asset
- $K$: Strike price of the option
- $T$: Time to expiration (in years)
- $r$: Risk-free interest rate (annualized)
- $\sigma$: Volatility of the underlying asset
- $N$: Number of time steps
- $\Delta t$: Length of each time step ($\Delta t = T/N$)

### Up and Down Factors

$$u = e^{\sigma \sqrt{\Delta t}}$$
$$d = e^{-\sigma \sqrt{\Delta t}} = 1/u$$

### Risk-Neutral Probability

$$p = \frac{e^{r\Delta t} - d}{u - d}$$

### Option Valuation

For a European call option, we calculate option payoffs at expiration: $\max(S_T - K, 0)$

Then we work backward through the tree using the risk-neutral valuation formula:

$$V_{i,j} = e^{-r\Delta t}[pV_{i+1,j+1} + (1-p)V_{i+1,j}]$$

## 2. Implementation in Python

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import seaborn as sns

# Set plotting style
plt.style.use('ggplot')
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 8)

In [ ]:
def binomial_european_call(S0, K, T, r, sigma, N):
    """Calculate European call option price using binomial model"""
    # Calculate time step
    dt = T/N
    
    # Calculate up and down factors
    u = np.exp(sigma * np.sqrt(dt))
    d = 1/u
    
    # Calculate risk-neutral probability
    p = (np.exp(r * dt) - d) / (u - d)
    
    # Initialize stock price tree
    stock_tree = np.zeros((N+1, N+1))
    
    # Populate the stock price tree
    for i in range(N+1):
        for j in range(i+1):
            stock_tree[i, j] = S0 * (u ** j) * (d ** (i - j))
    
    # Initialize option value tree
    option_tree = np.zeros((N+1, N+1))
    
    # Calculate call option payoffs at expiration
    for j in range(N+1):
        option_tree[N, j] = max(0, stock_tree[N, j] - K)
    
    # Backward induction to calculate option values
    for i in range(N-1, -1, -1):
        for j in range(i+1):
            option_tree[i, j] = np.exp(-r * dt) * (p * option_tree[i+1, j+1] + (1-p) * option_tree[i+1, j])
    
    return option_tree[0, 0], stock_tree, option_tree

In [ ]:
def plot_binomial_tree(stock_tree, option_tree, N):
    """Visualize binomial tree with stock prices and option values"""
    # Create a directed graph
    G = nx.DiGraph()
    
    # Add nodes and edges
    for i in range(N+1):
        for j in range(i+1):
            node_id = f"{i},{j}"
            stock_price = stock_tree[i, j]
            option_value = option_tree[i, j]
            
            G.add_node(node_id, pos=(i, j), stock_price=stock_price, option_value=option_value)
            
            if i < N:
                G.add_edge(node_id, f"{i+1},{j}")
                G.add_edge(node_id, f"{i+1},{j+1}")
    
    # Get positions for nodes
    pos = {}
    for node in G.nodes():
        i, j = map(int, node.split(','))
        pos[node] = (i, j - i/2)
    
    # Create figure with two subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))
    
    # Set titles
    fig.suptitle(f"European Call Option - Binomial Tree (N={N})")
    ax1.set_title("Stock Price Tree")
    ax2.set_title("Option Value Tree")
    
    # Draw edges
    nx.draw_networkx_edges(G, pos, ax=ax1, alpha=0.3)
    nx.draw_networkx_edges(G, pos, ax=ax2, alpha=0.3)
    
    # Draw nodes with color based on values
    node_colors1 = [stock_tree[int(node.split(',')[0]), int(node.split(',')[1])] for node in G.nodes()]
    nx.draw_networkx_nodes(G, pos, ax=ax1, node_size=700, node_color=node_colors1, cmap=plt.cm.viridis)
    
    node_colors2 = [option_tree[int(node.split(',')[0]), int(node.split(',')[1])] for node in G.nodes()]
    nx.draw_networkx_nodes(G, pos, ax=ax2, node_size=700, node_color=node_colors2, cmap=plt.cm.plasma)
    
    # Add colorbars
    sm1 = plt.cm.ScalarMappable(cmap=plt.cm.viridis, norm=plt.Normalize(vmin=min(node_colors1), vmax=max(node_colors1)))
    sm1.set_array([])
    plt.colorbar(sm1, ax=ax1).set_label('Stock Price')
    
    sm2 = plt.cm.ScalarMappable(cmap=plt.cm.plasma, norm=plt.Normalize(vmin=min(node_colors2), vmax=max(node_colors2)))
    sm2.set_array([])
    plt.colorbar(sm2, ax=ax2).set_label('Option Value')
    
    # Add labels
    labels1 = {node: f"${G.nodes[node]['stock_price']:.2f}" for node in G.nodes()}
    labels2 = {node: f"${G.nodes[node]['option_value']:.2f}" for node in G.nodes()}
    
    nx.draw_networkx_labels(G, pos, labels=labels1, ax=ax1, font_size=8)
    nx.draw_networkx_labels(G, pos, labels=labels2, ax=ax2, font_size=8)
    
    # Remove axes
    ax1.set_axis_off()
    ax2.set_axis_off()
    
    plt.tight_layout()
    plt.show()

## 3. Example: Pricing a European Call Option

In [ ]:
# Define parameters
S0 = 100    # Initial stock price
K = 100     # Strike price
T = 1       # Time to expiration (in years)
r = 0.05    # Risk-free interest rate
sigma = 0.2 # Volatility
N = 4       # Number of time steps

# Price the European call option
call_price, stock_tree, option_tree = binomial_european_call(S0, K, T, r, sigma, N)
print(f"European Call Option Price: ${call_price:.4f}")

# Visualize the binomial tree
plot_binomial_tree(stock_tree, option_tree, N)

## 4. Parameter Impact Analysis

Let's examine how different parameters affect the option price:

In [ ]:
def analyze_parameter_effect(param_name, param_range, S0, K, T, r, sigma, N):
    """Analyze how changing a parameter affects option price"""
    prices = []
    
    for value in param_range:
        params = {'S0': S0, 'K': K, 'T': T, 'r': r, 'sigma': sigma, 'N': N}
        params[param_name] = value
        
        if param_name == 'N':
            params['N'] = int(value)
            
        price, _, _ = binomial_european_call(params['S0'], params['K'], params['T'], 
                                           params['r'], params['sigma'], params['N'])
        prices.append(price)
    
    plt.figure(figsize=(8, 6))
    plt.plot(param_range, prices, 'o-', linewidth=2)
    
    param_labels = {
        'S0': 'Initial Stock Price ($)',
        'K': 'Strike Price ($)',
        'T': 'Time to Expiration (years)',
        'r': 'Risk-Free Rate',
        'sigma': 'Volatility',
        'N': 'Number of Time Steps'
    }
    
    plt.xlabel(param_labels.get(param_name, param_name))
    plt.ylabel('Option Price ($)')
    plt.title(f'Effect of {param_labels.get(param_name, param_name)} on Call Option Price')
    plt.grid(True)
    
    if param_name in ['r', 'sigma']:
        plt.gca().set_xticklabels([f'{x*100:.0f}%' for x in param_range])
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Vary stock price
analyze_parameter_effect('S0', np.linspace(80, 120, 10), S0, K, T, r, sigma, 20)

In [ ]:
# Vary volatility
analyze_parameter_effect('sigma', np.linspace(0.1, 0.5, 10), S0, K, T, r, sigma, 20)

In [ ]:
# Vary time to expiration
analyze_parameter_effect('T', np.linspace(0.1, 2, 10), S0, K, T, r, sigma, 20)

## 5. Conclusion

The binomial option pricing model provides a powerful and intuitive framework for valuing options. By constructing a tree of possible future stock prices and working backward to calculate option values, we can accurately price European call options.

Key insights from this implementation:

1. As the number of time steps increases, the model becomes more accurate
2. Higher volatility increases call option values due to greater upside potential
3. Longer time to expiration typically increases option values
4. The model can be extended to price various option types and incorporate dividends

This discrete-time approach also forms the foundation for understanding more complex option pricing models.